In [1]:
%pip install pandas numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from collections import defaultdict
import re
import os

In [3]:
# 1. Cargar el conjunto de datos 
csv_path = "spam_dataset_200.csv"
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"No se encontró {csv_path}. Asegúrate de que el archivo exista.")
df = pd.read_csv(csv_path)
print(f"Dataset cargado: {len(df)} mensajes")

Dataset cargado: 199 mensajes


In [4]:
# 2. Separar datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    df['message'], df['label'], test_size=0.3, random_state=42
)
print(f"Entrenamiento: {len(X_train)} mensajes, Prueba: {len(X_test)} mensajes")

Entrenamiento: 139 mensajes, Prueba: 60 mensajes


In [5]:
# 3. Implementar el clasificador Naive Bayes
class CustomNaiveBayes:
    def __init__(self):
        self.prior = {}  # P(C)
        self.likelihood = {}  # P(w|C)
        self.vocab = set()  # Vocabulario
        self.class_counts = defaultdict(int)  # Conteo de mensajes por clase
        self.word_counts = defaultdict(lambda: defaultdict(int))  # Conteo de palabras por clase
    
    def preprocess(self, text):
        """Convertir texto a minúsculas, eliminar caracteres especiales y tokenizar."""
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
        return text.split()
    
    def fit(self, messages, labels):
        """Calcular probabilidades a priori y condicionales."""
        # Calcular probabilidades a priori
        total_messages = len(labels)
        for label in set(labels):
            self.prior[label] = sum(1 for l in labels if l == label) / total_messages
        
        # Construir vocabulario y contar palabras
        for message, label in zip(messages, labels):
            self.class_counts[label] += 1
            words = self.preprocess(message)
            for word in words:
                self.word_counts[label][word] += 1
                self.vocab.add(word)
        
        # Calcular probabilidades condicionales con suavizado de Laplace
        for label in self.prior:
            self.likelihood[label] = {}
            total_words = sum(self.word_counts[label][word] for word in self.vocab)
            vocab_size = len(self.vocab)
            for word in self.vocab:
                self.likelihood[label][word] = (
                    (self.word_counts[label][word] + 1) / (total_words + vocab_size)
                )
    
    def calculate_posterior(self, message):
        """Calcular probabilidades posteriores P(C|M) para un mensaje."""
        words = self.preprocess(message)
        posteriors = {}
        
        # Calcular log-probabilidad para cada clase
        for label in self.prior:
            log_posterior = np.log(self.prior[label])
            for word in words:
                if word in self.vocab:
                    log_posterior += np.log(self.likelihood[label][word])
                else:
                    # Suavizado para palabras desconocidas
                    log_posterior += np.log(1 / (sum(self.word_counts[label].values()) + len(self.vocab)))
            posteriors[label] = log_posterior
        
        # Normalizar a probabilidades
        max_log = max(posteriors.values())
        exp_sums = sum(np.exp(log_p - max_log) for log_p in posteriors.values())
        normalized_posteriors = {
            label: np.exp(log_p - max_log) / exp_sums for label, log_p in posteriors.items()
        }
        
        return normalized_posteriors
    
    def predict(self, messages):
        """Predecir la clase y devolver probabilidades posteriores para cada mensaje."""
        predictions = []
        posterior_probs = []
        
        for message in messages:
            posteriors = self.calculate_posterior(message)
            predicted_label = max(posteriors, key=posteriors.get)
            predictions.append(predicted_label)
            posterior_probs.append(posteriors)
        
        return predictions, posterior_probs

In [6]:
# 4. Entrenar y evaluar el clasificador personalizado
custom_nb = CustomNaiveBayes()
custom_nb.fit(X_train, y_train)
custom_predictions, custom_posteriors = custom_nb.predict(X_test)

# Mostrar probabilidades posteriores para los primeros 5 mensajes
print("\nProbabilidades posteriores del clasificador personalizado (primeros 5 mensajes):")
for i, (message, pred, probs) in enumerate(zip(X_test[:5], custom_predictions[:5], custom_posteriors[:5])):
    print(f"\nMensaje {i+1}: {message}")
    print(f"Predicción: {pred}")
    print(f"Probabilidades posteriores: {probs}")


Probabilidades posteriores del clasificador personalizado (primeros 5 mensajes):

Mensaje 1: Descuento del 35% en tu compra
Predicción: spam
Probabilidades posteriores: {'spam': np.float64(0.9904198501169023), 'no_spam': np.float64(0.009580149883097664)}

Mensaje 2: Felicidades, ganaste un iPhone
Predicción: spam
Probabilidades posteriores: {'spam': np.float64(0.9980774812050127), 'no_spam': np.float64(0.0019225187949871928)}

Mensaje 3: Almuerzo en factura este reunión
Predicción: no_spam
Probabilidades posteriores: {'spam': np.float64(7.446141856409593e-05), 'no_spam': np.float64(0.999925538581436)}

Mensaje 4: Tu casa ha sido enviado
Predicción: no_spam
Probabilidades posteriores: {'spam': np.float64(0.00010211569266414951), 'no_spam': np.float64(0.9998978843073357)}

Mensaje 5: Última chance para oferta de coche
Predicción: spam
Probabilidades posteriores: {'spam': np.float64(0.9986857001446129), 'no_spam': np.float64(0.0013142998553871358)}


In [7]:
# 5. Comparar con scikit-learn
# Vectorización de los mensajes
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Entrenar modelo de scikit-learn
sklearn_nb = MultinomialNB()
sklearn_nb.fit(X_train_vec, y_train)
sklearn_predictions = sklearn_nb.predict(X_test_vec)
sklearn_posteriors = sklearn_nb.predict_proba(X_test_vec)

# Mostrar probabilidades posteriores de scikit-learn para los primeros 5 mensajes
print("\nProbabilidades posteriores de scikit-learn (primeros 5 mensajes):")
for i, (message, pred, probs) in enumerate(zip(X_test[:5], sklearn_predictions[:5], sklearn_posteriors[:5])):
    print(f"\nMensaje {i+1}: {message}")
    print(f"Predicción: {pred}")
    print(f"Probabilidades posteriores: {{'no_spam': {probs[0]:.4f}, 'spam': {probs[1]:.4f}}}")

# Mostrar métricas de evaluación
print("\nResultados del clasificador personalizado:")
print("Precisión:", accuracy_score(y_test, custom_predictions))
print("\nReporte de clasificación:")
print(classification_report(y_test, custom_predictions))

print("\nResultados del clasificador de scikit-learn:")
print("Precisión:", accuracy_score(y_test, sklearn_predictions))
print("\nReporte de clasificación:")
print(classification_report(y_test, sklearn_predictions))


Probabilidades posteriores de scikit-learn (primeros 5 mensajes):

Mensaje 1: Descuento del 35% en tu compra
Predicción: spam
Probabilidades posteriores: {'no_spam': 0.0070, 'spam': 0.9930}

Mensaje 2: Felicidades, ganaste un iPhone
Predicción: spam
Probabilidades posteriores: {'no_spam': 0.0018, 'spam': 0.9982}

Mensaje 3: Almuerzo en factura este reunión
Predicción: no_spam
Probabilidades posteriores: {'no_spam': 0.9999, 'spam': 0.0001}

Mensaje 4: Tu casa ha sido enviado
Predicción: no_spam
Probabilidades posteriores: {'no_spam': 0.9999, 'spam': 0.0001}

Mensaje 5: Última chance para oferta de coche
Predicción: spam
Probabilidades posteriores: {'no_spam': 0.0012, 'spam': 0.9988}

Resultados del clasificador personalizado:
Precisión: 1.0

Reporte de clasificación:
              precision    recall  f1-score   support

     no_spam       1.00      1.00      1.00        31
        spam       1.00      1.00      1.00        29

    accuracy                           1.00        60
   m